# CS Netflix Dataset Engineering

reference:
[tutorial!](https://www.samuelliedtke.com/blog/explore-imdb-dataset-jupyter-notebook-and-pandas)

In [1]:
import pandas as pd

In [2]:
#netflix data
netflix = pd.read_csv('netflix_clean.csv')

In [3]:
#all from tutorial page
import gzip
import shutil
 
with gzip.open('title.basics.tsv.gz', 'rb') as f_in:
    with open('title.basics.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
with gzip.open('title.ratings.tsv.gz', 'rb') as f_in:
    with open('title.ratings.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)


In [4]:
#all from tutorial page
basics = pd.read_csv('title.basics.tsv', sep='\t',
    low_memory=False, na_values=['\\N'])
ratings = pd.read_csv('title.ratings.tsv', sep='\t',
    low_memory=False, na_values=['\\N'])

In [5]:
print(len(basics), len(ratings))

10759885 1435757


In [6]:
basics.titleType.value_counts()

titleType
tvEpisode       8236256
short            993497
movie            679989
video            291910
tvSeries         262560
tvMovie          145415
tvMiniSeries      54179
tvSpecial         47559
videoGame         38214
tvShort           10305
tvPilot               1
Name: count, dtype: int64

In [7]:
#combining imdb data, after changing my selection algo didn't need comment
imdb = pd.merge(basics, ratings, on='tconst')

In [8]:
#Ensuring no data with different capitalisation is missed
imdb.primaryTitle = imdb.primaryTitle.str.lower()
netflix.title = netflix.title.str.lower()

# DataSet 1; merging Netflix and IMDB data

**6625 entries with most voted duplicates**

**4484 entries with clean data**

**4490 entries**


**Take most rated duplicate after merge**


Removed any duplicated items after merge, removed any data missing vote/rating entries.

In [9]:
# imdb.titleType.value_counts()

In [10]:
# imdb.head()

In [11]:
imdb=imdb[imdb.titleType.isin(['movie','tvSeries','tvMovie','tvMiniSeries'])]

In [12]:
#merging netflix and the combined imdb data, couldn't find a better key than title
#even making a new key didn't work (title_year) since the year of release was different in netflix and imdb sets for several shows
net_imdb = pd.merge(netflix,imdb, left_on='title',right_on='primaryTitle', how='left')

In [13]:
#many movie/tv titles are the same in the imdb data: strategy take only most reviewed
#getting rid of the duplicate entries
duplicates = net_imdb[net_imdb.title.duplicated(keep =False)]

dupes_i = duplicates.index
net_imdb = net_imdb.drop(dupes_i)

#writing the most reviewed data from duplicates
max_votes_dupes = duplicates.groupby('title')['numVotes'].max()

duplicates['title_votes'] = duplicates.title.astype(str) + '_' + duplicates.numVotes.astype(str)

key = max_votes_dupes.index + '_' + max_votes_dupes.astype(str)
duplicates
duplicates=duplicates[duplicates.title_votes.isin(key)]

net_imdb = pd.concat([net_imdb, duplicates], axis=0)
# net_imdb

In [14]:
net_imdb.reset_index(drop=True, inplace=True)
net_imdb.drop(['tconst',	'titleType',	'primaryTitle',	'originalTitle',	'isAdult',	'startYear',	'endYear',	'runtimeMinutes',	'genres','title_votes'],axis=1, inplace=True)

In [15]:
net_imdb = net_imdb[~net_imdb.numVotes.isna()]

In [16]:
# code for just removing all dupes
# dupes_i = duplicates.index
# net_imdb = net_imdb.drop(dupes_i)
# net_imdb = net_imdb.reset_index(drop=True)

In [17]:
# display(net_imdb.head(0), imdb.head(0), netflix.head(0))

In [18]:
len(net_imdb)

6892

In [19]:
net_imdb.rename({'averageRating':'imdb_score','numVotes':'imdb_votes'},axis=1, inplace = True)

In [20]:
net_imdb.to_csv('netflix_imdb.csv', index=False)

### _______________________________________________________________________________________
### Bellow used to try and maximise no. entries
### _______________________________________________________________________________________

However!
It assumes the netflix item is the most voted imdb item; not necessarily the case: stick to other datasets.

In [21]:
# #take the most voted for items from duplicate list
# groups = duplicates.groupby('title')['numVotes'].max()
# groups = groups.reset_index()

# #create a unique key for finding the most voted items in our merged df
# groups['title_votes'] = groups.title.astype(str) +'_'+ groups.numVotes.astype(str)
# og_imdb['title_votes'] = og_imdb.title.astype(str) + '_' + og_imdb.numVotes.astype(str)
# uniques = og_imdb[og_imdb.title_votes.isin(groups.title_votes)]

In [22]:
# #get rid of all duplicates and concatenate this data with the uniques.
# dupes_i = duplicates.index
# og_imdb_duped=og_imdb.drop(dupes_i, axis=0)
# final = pd.concat([og_imdb_duped, uniques],axis=0)

In [23]:
# final.info()

## DataSet 2: IMDB top 10k most popular media

In [24]:
imdb_10k_votes = imdb.sort_values('numVotes', ascending=False)[:100000]

In [25]:
len(imdb_10k_votes)

100000

In [26]:
imdb_10k_votes.to_csv('imdb_10k_votes.csv', index=False)

## Dataset 3: Merging netflix and kaggle imdb data
**Only 3785 entries**

In [27]:
netflix = pd.read_csv('netflix_clean.csv')

In [28]:
imdb_kaggle = pd.read_csv('Netflix Tv Shows and Movies.csv', index_col = 'index')

In [29]:
imdb_kaggle.drop(['id','type','description','release_year','age_certification','runtime', 'imdb_id'], axis=1, inplace=True)

In [30]:
# display(netflix.head(), imdb_kaggle.head())

In [31]:
net_ikag = pd.merge(netflix, imdb_kaggle, on='title', how='left')

In [32]:
net_ikag = net_ikag[~net_ikag.imdb_votes.isna()].reset_index(drop=True)

In [33]:
len(net_ikag)

3785

In [34]:
net_ikag.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,month_added,year_added,imdb_score,imdb_votes
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,2021-09-25,2020.0,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",9.0,2021.0,7.4,6390.0


In [35]:
net_ikag.to_csv('netflix_imdb_kaggle.csv', index=False)

# Dataset 4: combining 1 and 3??

**7079 with taking most voted duplicates**

**6007 entries with clean data**

**6013 entries**

This is a great work-around, just worth double checking a couple entries from ikag to make sure the kaggle set is good.

In [36]:
net_ikag.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description',
       'month_added', 'year_added', 'imdb_score', 'imdb_votes'],
      dtype='object')

In [37]:
net_ikag.title = net_ikag.title.str.lower()
net_imdb.title = net_imdb.title.str.lower()

In [38]:
net_icomb = pd.concat([net_imdb, net_ikag],axis=0, ignore_index=True)

In [39]:
dup_index = net_icomb[net_icomb.title.duplicated()].index
net_icomb.drop(dup_index, axis=0, inplace =True)

In [40]:
net_icomb['id'] = net_icomb.show_id.str.strip('s').astype(int)
net_icomb.sort_values('id')
net_icomb.reset_index(drop=True, inplace =True)
net_icomb.drop('id',axis=1, inplace = True)

In [41]:
len(net_icomb)

7298

In [42]:
net_icomb.to_csv('netflix_both.csv', index=False)

In [43]:
net_icomb[net_icomb.title.str.startswith('frien')]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,month_added,year_added,imdb_score,imdb_votes
787,s1390,Movie,friendsgiving,Nicol Paone,"Malin Akerman, Kat Dennings, Aisha Tyler, Jane...",United States,2021-01-21,2020.0,R,95 min,Comedies,A quiet holiday dinner among friends turns int...,1.0,2021.0,4.5,3760.0
2411,s4201,TV Show,friends from college,Unknown,"Keegan-Michael Key, Cobie Smulders, Billy Eich...",United States,2019-01-11,2019.0,TV-MA,2 Seasons,"Romantic TV Shows, TV Comedies","Twenty years after graduation, a tight-knit gr...",1.0,2019.0,6.9,23414.0
5549,s2724,Movie,friendship,Raj Khosla,"Amitabh Bachchan, Shatrughan Sinha, Zeenat Ama...",India,2020-04-01,1980.0,TV-14,161 min,"Action & Adventure, Dramas, International Movies","Best friends since childhood, a righteous cop ...",4.0,2020.0,6.4,1108.0
6416,s6806,Movie,friend request,Simon Verhoeven,"Alycia Debnam-Carey, William Moseley, Connor P...",Germany,2018-04-18,2016.0,R,92 min,"Horror Movies, International Movies",A popular college student's love of social med...,4.0,2018.0,5.2,27831.0
6417,s6807,TV Show,friends,Unknown,"Jennifer Aniston, Courteney Cox, Lisa Kudrow, ...",United States,NaN,2003.0,TV-14,10 Seasons,"Classic & Cult TV, TV Comedies",This hit sitcom follows the merry misadventure...,NaN,NaN,8.9,1089437.0


## DataSet 5: entries in the top 10k imdb not in netflix data

In [44]:
imdb_not_netflix = imdb_10k_votes[~imdb_10k_votes.primaryTitle.isin(netflix.title.str.lower())]
imdb_not_netflix.reset_index(drop=True, inplace=True)
imdb_not_netflix.to_csv('imdb_not_netflix.csv', index=False)

In [45]:
netflix

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,month_added,year_added
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,2021-09-25,2020.0,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",9.0,2021.0
1,s2,TV Show,Blood & Water,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021.0,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",9.0,2021.0
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Unknown,2021-09-24,2021.0,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,9.0,2021.0
3,s4,TV Show,Jailbirds New Orleans,Unknown,Unknown,Unknown,2021-09-24,2021.0,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",9.0,2021.0
4,s5,TV Show,Kota Factory,Unknown,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021.0,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,9.0,2021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,2019-11-20,2007.0,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a...",11.0,2019.0
8803,s8804,TV Show,Zombie Dumb,Unknown,Unknown,Unknown,2019-07-01,2018.0,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g...",7.0,2019.0
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,2019-11-01,2009.0,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...,11.0,2019.0
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,2020-01-11,2006.0,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero...",1.0,2020.0


In [46]:
imdb_10k_votes

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
83560,tt0111161,movie,the shawshank redemption,The Shawshank Redemption,0.0,1994.0,NaN,142,Drama,9.3,2892098
254824,tt0468569,movie,the dark knight,The Dark Knight,0.0,2008.0,NaN,152,"Action,Crime,Drama",9.0,2873906
662834,tt1375666,movie,inception,Inception,0.0,2010.0,NaN,148,"Action,Adventure,Sci-Fi",8.8,2553824
100258,tt0137523,movie,fight club,Fight Club,0.0,1999.0,NaN,139,Drama,8.8,2326975
444076,tt0944947,tvSeries,game of thrones,Game of Thrones,0.0,2011.0,2019.0,60,"Action,Adventure,Drama",9.2,2287976
...,...,...,...,...,...,...,...,...,...,...,...
1013732,tt2796792,tvMovie,a way back home,Shuffleton's Barbershop,0.0,2013.0,NaN,84,Drama,5.9,339
1270689,tt6236404,movie,kadavul irukaan kumaru,Kadavul Irukaan Kumaru,0.0,2016.0,NaN,138,Comedy,3.1,339
63745,tt0087888,movie,"partir, revenir","Partir, revenir",0.0,1985.0,NaN,114,"Drama,War",6.3,339
36912,tt0056995,movie,to bed or not to bed,Il diavolo,0.0,1963.0,NaN,103,"Comedy,Fantasy",6.3,339
